In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import re 
from scipy import sparse
import time
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_colwidth=300
pd.options.display.max_columns = 100

from sklearn.linear_model import Ridge

In [2]:
import re
import sys
import unicodedata

from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer

import emoji

CUSTOM_TABLE = str.maketrans(
    {
        "\xad": None,
        "\x7f": None,
        "\ufeff": None,
        "\u200b": None,
        "\u200e": None,
        "\u202a": None,
        "\u202c": None,
        "‘": "'",
        "’": "'",
        "`": "'",
        "“": '"',
        "”": '"',
        "«": '"',
        "»": '"',
        "ɢ": "G",
        "ɪ": "I",
        "ɴ": "N",
        "ʀ": "R",
        "ʏ": "Y",
        "ʙ": "B",
        "ʜ": "H",
        "ʟ": "L",
        "ғ": "F",
        "ᴀ": "A",
        "ᴄ": "C",
        "ᴅ": "D",
        "ᴇ": "E",
        "ᴊ": "J",
        "ᴋ": "K",
        "ᴍ": "M",
        "Μ": "M",
        "ᴏ": "O",
        "ᴘ": "P",
        "ᴛ": "T",
        "ᴜ": "U",
        "ᴡ": "W",
        "ᴠ": "V",
        "ĸ": "K",
        "в": "B",
        "м": "M",
        "н": "H",
        "т": "T",
        "ѕ": "S",
        "—": "-",
        "–": "-",
    }
)

WORDS_REPLACER = [
    ("sh*t", "shit"),
    ("s**t", "shit"),
    ("f*ck", "fuck"),
    ("fu*k", "fuck"),
    ("f**k", "fuck"),
    ("f*****g", "fucking"),
    ("f***ing", "fucking"),
    ("f**king", "fucking"),
    ("p*ssy", "pussy"),
    ("p***y", "pussy"),
    ("pu**y", "pussy"),
    ("p*ss", "piss"),
    ("b*tch", "bitch"),
    ("bit*h", "bitch"),
    ("h*ll", "hell"),
    ("h**l", "hell"),
    ("cr*p", "crap"),
    ("d*mn", "damn"),
    ("stu*pid", "stupid"),
    ("st*pid", "stupid"),
    ("n*gger", "nigger"),
    ("n***ga", "nigger"),
    ("f*ggot", "faggot"),
    ("scr*w", "screw"),
    ("pr*ck", "prick"),
    ("g*d", "god"),
    ("s*x", "sex"),
    ("a*s", "ass"),
    ("a**hole", "asshole"),
    ("a***ole", "asshole"),
    ("a**", "ass"),
]

REGEX_REPLACER = [
    (re.compile(pat.replace("*", "\*"), flags=re.IGNORECASE), repl)
    for pat, repl in WORDS_REPLACER
]

RE_SPACE = re.compile(r"\s")
RE_MULTI_SPACE = re.compile(r"\s+")

NMS_TABLE = dict.fromkeys(
    i for i in range(sys.maxunicode + 1) if unicodedata.category(chr(i)) == "Mn"
)

HEBREW_TABLE = {i: "א" for i in range(0x0590, 0x05FF)}
ARABIC_TABLE = {i: "ا" for i in range(0x0600, 0x06FF)}
CHINESE_TABLE = {i: "是" for i in range(0x4E00, 0x9FFF)}
KANJI_TABLE = {i: "ッ" for i in range(0x2E80, 0x2FD5)}
HIRAGANA_TABLE = {i: "ッ" for i in range(0x3041, 0x3096)}
KATAKANA_TABLE = {i: "ッ" for i in range(0x30A0, 0x30FF)}

TABLE = dict()
TABLE.update(CUSTOM_TABLE)
TABLE.update(NMS_TABLE)
# Non-english languages
TABLE.update(CHINESE_TABLE)
TABLE.update(HEBREW_TABLE)
TABLE.update(ARABIC_TABLE)
TABLE.update(HIRAGANA_TABLE)
TABLE.update(KATAKANA_TABLE)
TABLE.update(KANJI_TABLE)


EMOJI_REGEXP = emoji.get_emoji_regexp()

UNICODE_EMOJI_MY = {
    k: f" EMJ {v.strip(':').replace('_', ' ')} "
    for k, v in emoji.UNICODE_EMOJI_ALIAS_ENGLISH.items()
}


def my_demojize(string: str) -> str:
    """
    Modified version of the demojize function from emoji
    """

    def replace(match):
        return UNICODE_EMOJI_MY.get(match.group(0), match.group(0))

    return re.sub("\ufe0f", "", EMOJI_REGEXP.sub(replace, string))


def fix_tokens(tokens):
    """
    Expects a list of lower-cased tokens from TweeterTokenizer
    """
    for token in tokens:
        if token == "gov't" or token == "govt":
            yield "government"
        elif token == "i'm":
            yield "i"
            yield "am"
        elif token.endswith("n't"):
            yield token[:-3]
            yield "not"
        elif token.endswith("'re"):
            yield token[:-3]
            yield "are"
        elif token.endswith("'ll"):
            yield token[:-3]
            yield "will"
        elif token.endswith("'ve"):
            yield token[:-3]
            yield "have"
        elif token.endswith("'s"):
            yield token[:-2]
            yield "'s"
        else:
            yield token


def normalize(text: str) -> str:
    text = my_demojize(text)
    text = RE_SPACE.sub(" ", text)
    text = unicodedata.normalize("NFKD", text)
    text = text.translate(TABLE)
    text = RE_MULTI_SPACE.sub(" ", text).strip()

    for pattern, repl in REGEX_REPLACER:
        text = pattern.sub(repl, text)

    return text


In [3]:


df = pd.read_csv('/home/xiaoguzai/数据/kaggle有毒评论对比/ruddit/ruddit_with_text.csv')
df.head()



,post_id,comment_id,txt,url,offensiveness_score
0,42g75o,cza1q49,"> The difference in average earnings between men and women can be explained by taking into account relevant factors.\n\nSo it isn't a myth, you just feel that you can explain it.",https://www.reddit.com/r/changemyview/comments/42g75o/cmv_the_so_called_wage_gap_is_a_myth//cza1q49/,-0.083
1,42g75o,cza1wdh,"The myth is that the ""gap"" is entirely based on the sex of the person.",https://www.reddit.com/r/changemyview/comments/42g75o/cmv_the_so_called_wage_gap_is_a_myth//cza1wdh/,-0.022
2,42g75o,cza23qx,[deleted],https://www.reddit.com/r/changemyview/comments/42g75o/cmv_the_so_called_wage_gap_is_a_myth//cza23qx/,0.167
3,42g75o,cza2bw8,"The assertion is that women get paid less for the *same* jobs, and that they get paid less *because* they are women.",https://www.reddit.com/r/changemyview/comments/42g75o/cmv_the_so_called_wage_gap_is_a_myth//cza2bw8/,-0.146
4,42g75o,cza2iji,"You said in the OP that's not what they're measuring. They're measuring aggregate. \n\nAs entertainers, women make less. Isn't that what JLaw was complaining about?",https://www.reddit.com/r/changemyview/comments/42g75o/cmv_the_so_called_wage_gap_is_a_myth//cza2iji/,-0.083


In [6]:
df['y'] = df['offensiveness_score']
df['text'] = df['txt']

In [7]:
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string


    # text = text.lower()
    # Clean some punctutations
    text = text.replace('\n', ' \n ')
    # Remove all numbers
    text = re.sub(r'[0-9]', '', text)
    # Add space between pattern
    text = re.sub(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3', text)
    # Replace repeating puncts more than 3 times to length of 3
    text = re.sub(r'([*!?\'])\1\1{2,}',r'\1\1\1', text)    
    # Add space around repeating puncts
    text = re.sub(r'([*!?\']+)',r' \1 ', text)    
    # patterns with repeating characters 
    # naaaahhhhhhh to naaaahh
    text = re.sub(r'([a-zA-Z])\1{2,}\b',r'\1\1', text)
    # naaaahh to naaahh
    text = re.sub(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1', text)
    # multiple spaces to one
    text = re.sub(r'[ ]{2,}',' ', text).strip()   
    text = normalize(text)


    return text

df = df.dropna(axis = 0)

vec = TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb', ngram_range = (3,5), max_features = 46000)
vec.fit(df['text'])



TfidfVectorizer(analyzer='char_wb', max_df=0.5, max_features=46000, min_df=3,
                ngram_range=(3, 5))

In [12]:
from gensim.models import KeyedVectors, FastText

fmodel = FastText.load('/home/xiaoguzai/模型/unbiased-toxic-roberta/Jigsaw-Fasttext-Word-Embeddings-256D.bin')

from scipy.sparse import hstack

def splitter(text):
    tokens = []
    
    for word in text.split(' '):
        tokens.append(word)
    
    return tokens

def vectorizer(text):
    tokens = splitter(text)
    
    x1 = vec.transform([text]).toarray()
    x2 = np.mean(fmodel.wv[tokens], axis = 0).reshape(1, -1)
    x = np.concatenate([x1, x2], axis = -1).astype(np.float16)
    del x1
    del x2 
    
    return x   

X_list = []

for text in df.text:
    X_list.append(vectorizer(text))

EMB_DIM = len(vec.vocabulary_) + 256

X_np = np.array(X_list).reshape(-1, EMB_DIM)

from scipy import sparse

X = sparse.csr_matrix(X_np)
del X_np


model = Ridge(alpha=0.5)
model.fit(X, df['y'])


l_model = Ridge(alpha=1.)
l_model.fit(X, df['y'])


d_model = Ridge(alpha=1.5)
d_model.fit(X, df['y'])


s_model = Ridge(alpha=2.)
s_model.fit(X, df['y'])


import pickle

with open('model.pkl','wb') as f:
    pickle.dump(model,f) 
with open('lmodel.pkl','wb') as f:
    pickle.dump(l_model,f)
with open('dmodel.pkl','wb') as f:
    pickle.dump(d_model,f)
with open('smodel.pkl','wb') as f:
    pickle.dump(s_model,f)

# import pickle

# with open('../input/fasttext-ridge/model.pkl','rb') as f:
#     model = pickle.load(f)
# with open('../input/fasttext-ridge/lmodel.pkl','rb') as f:
#     l_model = pickle.load(f)
# with open('../input/fasttext-ridge/dmodel.pkl','rb') as f:
#     d_model = pickle.load(f)
# with open('../input/fasttext-ridge/smodel.pkl','rb') as f:
#     s_model = pickle.load(f)

df_val = pd.read_csv("/home/xiaoguzai/数据/Kaggle Jigsaw Rate Severity of Toxic Comments/validation_true.csv")

# df_val = pd.read_csv("/content/drive/Shareddrives/Jigsaw/validation_true.csv")

tqdm.pandas()
df_val['less_toxic'] = df_val['less_toxic'].progress_apply(text_cleaning)
df_val['more_toxic'] = df_val['more_toxic'].progress_apply(text_cleaning)


df_val.to_csv("cleaned_valid4.csv")

X_less_toxic_temp = []

for text in df_val.less_toxic:
    X_less_toxic_temp.append(vectorizer(text))

X_less_toxic_temp = np.array(X_less_toxic_temp).reshape(-1, EMB_DIM)
X_less_toxic = sparse.csr_matrix(X_less_toxic_temp)

del X_less_toxic_temp
    
X_more_toxic_temp = []

for text in df_val.more_toxic:
    X_more_toxic_temp.append(vectorizer(text))
    
X_more_toxic_temp = np.array(X_more_toxic_temp).reshape(-1, EMB_DIM)
X_more_toxic = sparse.csr_matrix(X_more_toxic_temp)

del X_more_toxic_temp

less1 = model.predict(X_less_toxic)
more1 = model.predict(X_more_toxic)
# Validation Accuracy
print((less1 < more1).mean())


less2  = l_model.predict(X_less_toxic)
more2 = l_model.predict(X_more_toxic)
# Validation Accuracy
print((less2 < more2).mean())


less3 = d_model.predict(X_less_toxic)
more3 = d_model.predict(X_more_toxic)
# Validation Accuracy
print((less3 < more3).mean())

less4 = s_model.predict(X_less_toxic)
more4 = s_model.predict(X_more_toxic)
# Validation Accuracy
print((less4 < more4).mean())



FileNotFoundError: [Errno 2] No such file or directory: '/home/xiaoguzai/模型/unbiased-toxic-roberta/Jigsaw-Fasttext-Word-Embeddings-256D.bin.wv.vectors_ngrams.npy'